In [1]:
import json
import requests
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df1 = pd.read_excel("S&P 500 Index.xlsx",header=None)
df1.shape

(500, 1)

In [3]:
new_columns = ["index", "code", "name"]
df1[new_columns] = df1.apply(lambda x: x[0].split(" "), axis = 1, result_type = "expand")
df1.shape

(500, 4)

In [4]:
stock_list = df1.code.to_list()
stock_list[:5]

['A', 'AA', 'AAPL', 'ABC', 'ABT']

In [5]:
len(stock_list)

500

In [ ]:
api_key = YOUR_API_KEY  # You may find your api key here https://site.financialmodelingprep.com/developer/docs/api-keys

In [21]:
all = pd.DataFrame()
for stock in tqdm(stock_list):
    for page in tqdm(range(500)):
        url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={stock}&page={page+1}&apikey={api_key}"
        res = requests.get(url)
        res = json.loads(res.text)
        if len(res) == 0:
            break
        else:
            res = pd.DataFrame(res)
            all = pd.concat([all, res])

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [40]:
all_other = pd.DataFrame()
for stock in tqdm(other_code_list):
    for page in tqdm(range(500),desc = stock):
        url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={stock}&page={page+1}&apikey=bbefd29f7b576e2db216612f20b7c39c"
        res = requests.get(url)
        res = json.loads(res.text)
        if len(res) == 0:
            break
        else:
            res = pd.DataFrame(res)
            all_other = pd.concat([all_other, res])

  0%|          | 0/454 [00:00<?, ?it/s]

A:   0%|          | 0/500 [00:00<?, ?it/s]

AA:   0%|          | 0/500 [00:00<?, ?it/s]

ABC:   0%|          | 0/500 [00:00<?, ?it/s]

ABT:   0%|          | 0/500 [00:00<?, ?it/s]

...

XRX:   0%|          | 0/500 [00:00<?, ?it/s]

YHOO:   0%|          | 0/500 [00:00<?, ?it/s]

YUM:   0%|          | 0/500 [00:00<?, ?it/s]

ZION:   0%|          | 0/500 [00:00<?, ?it/s]

ZMH:   0%|          | 0/500 [00:00<?, ?it/s]

In [22]:
all.shape

(244667, 7)

In [25]:
all = all.reset_index(drop=True)

In [26]:
all

,symbol,publishedDate,title,image,site,text,url
0,AAL,2022-06-06 14:06:29,"In The Partnership With JetBlue, Investors Sho...",https://cdn.snapi.dev/images/v1/q/g/image-6917...,Seeking Alpha,The U.S. airline industry is experiencing stro...,https://seekingalpha.com/article/4516691-in-pa...
1,AAL,2022-06-03 18:01:04,American CEO says the airline has grounded 100...,https://cdn.snapi.dev/images/v1/6/2/62101889f0...,Business Insider,American is the latest carrier to take action ...,https://www.businessinsider.com/american-groun...
2,AAL,2022-06-03 17:33:35,"Starbucks Shanghai stores reopen, American Air...",https://cdn.snapi.dev/images/v1/b/6/starbucks-...,Yahoo Finance,Yahoo Finance Live checks out several stocks t...,https://www.youtube.com/watch?v=wyo2MRzH7FU
3,AAL,2022-06-03 16:05:44,Why American Airlines Stock Was Diving Today,https://cdn.snapi.dev/images/v1/h/k/qjne2222-1...,The Motley Fool,An important cost item for the company will be...,https://www.fool.com/investing/2022/06/03/why-...
4,AAL,2022-06-03 14:49:02,"American Airlines Raises Outlook, Falls On Fue...",https://cdn.snapi.dev/images/v1/n/v/social-131...,Benzinga,"American Airlines Group, Inc (NASDAQ: AAL) was...",https://www.benzinga.com/trading-ideas/movers/...
...,...,...,...,...,...,...,...
244662,XEL,2019-04-25 14:58:08,Xcel Energy Inc. (XEL) CEO Ben Fowke on Q1 201...,https://cdn.snapi.dev/images/v1/7/q/transcript...,Seeking Alpha,Xcel Energy Inc. (XEL) CEO Ben Fowke on Q1 201...,https://seekingalpha.com/article/4256807-xcel-...
244663,XEL,2019-04-25 09:26:00,Xcel Energy's (XEL) Earnings Meet Estimates in...,https://cdn.snapi.dev/images/v1/f/k/utilities8...,Zacks Investment Research,New electric and natural gas rate drives Xcel ...,https://www.zacks.com/stock/news/399436/xcel-e...
244664,XEL,2019-04-25 08:29:00,"Xcel Energy (XEL) Q1 Earnings Meet Estimates, ...",https://cdn.snapi.dev/images/v1/c/k/utilities6...,Zacks Investment Research,Higher revenues in all three segments boost Xc...,https://www.zacks.com/stock/news/399179/xcel-e...
244665,XEL,2019-04-22 08:30:00,Xcel Energy (XEL) to Post Q1 Earnings: What's ...,https://cdn.snapi.dev/images/v1/h/y/utilities4...,Zacks Investment Research,Positive impact of steel for fuel investment s...,https://www.zacks.com/stock/news/389861/xcel-e...


In [27]:
all.to_csv("dataset.csv")

In [28]:
all.symbol.nunique()

80

In [34]:
all.groupby("symbol").publishedDate.min().to_csv("last_date.csv")

In [32]:
all.head(2)

,symbol,publishedDate,title,image,site,text,url
0,AAL,2022-06-06 14:06:29,"In The Partnership With JetBlue, Investors Sho...",https://cdn.snapi.dev/images/v1/q/g/image-6917...,Seeking Alpha,The U.S. airline industry is experiencing stro...,https://seekingalpha.com/article/4516691-in-pa...
1,AAL,2022-06-03 18:01:04,American CEO says the airline has grounded 100...,https://cdn.snapi.dev/images/v1/6/2/62101889f0...,Business Insider,American is the latest carrier to take action ...,https://www.businessinsider.com/american-groun...


In [13]:
res.url[0]

'https://www.youtube.com/watch?v=IL91F3dTpRs&t=124s'